In [ ]:
# ===============================
# 🔹 STEP 1: Environment Setup
# ===============================
!nvidia-smi   # check GPU (Tesla T4 / A100 is ideal)

!pip install ultralytics roboflow gradio opencv-python --quiet

import os
import cv2
from IPython.display import Image, display
from ultralytics import YOLO


Fri Sep  5 04:42:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# ===============================
# 🔹 STEP 2: Dataset Download (Roboflow)
# ===============================
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="abcdefeeghijklmn")
project = rf.workspace("adeghjkl;lkjhgfds").project("mineral-identification-870uk")
version = project.version(8)
dataset = version.download("yolov8")

# Path where data.yaml is stored
dataset_path = dataset.location
print("Dataset path:", dataset_path)
!ls {dataset_path}


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Mineral-Identification-8 in yolov8:: 100%|██████████| 16012/16012 [00:02<00:00, 7371.31it/s]


Dataset path: /content/Mineral-Identification-8
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [ ]:
# ===============================
# 🔹 STEP 3: Mount Google Drive (for saving weights)
# ===============================
from google.colab import drive
drive.mount('/content/drive')

drive_save_path = "/content/drive/MyDrive/mineral_detection"
os.makedirs(drive_save_path, exist_ok=True)
print("Google Drive path:", drive_save_path)


In [4]:
# ===============================
# Use yolov8m or yolov8l for stronger accuracy on larger datasets
model = YOLO("yolov8m.pt")   # pretrained on COCO



# ===============================
# 🔹 STEP 2: Train with Strong Settings
# ===============================
results = model.train(
    data=f"{dataset_path}/data.yaml",   # dataset config file
    epochs=50,                         # train longer for convergence
    imgsz=640,                          # input image size (try 768 if GPU strong)
    batch=32,                           # larger batch = more stable training
    optimizer="AdamW",                  # better optimizer than SGD
    lr0=0.001,                          # initial learning rate
    lrf=0.1,                            # final learning rate multiplier
    momentum=0.937,                     # optimizer momentum
    weight_decay=0.0005,                # L2 regularization
    warmup_epochs=5,                    # gradual warmup
    patience=10,                        # early stopping
    mixup=0.2, mosaic=1.0,              # augmentations
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # color augmentations
    degrees=10, translate=0.1, scale=0.5, shear=0.1,  # geometric augmentations
    project="mineral-detection",
    name="yolov8m-advanced",
    plots=True,
    verbose=True
)

# ===============================
# 🔹 STEP 3: Evaluate the Model
# ===============================
metrics = model.val()
print("📊 Evaluation Metrics:", metrics)

# Training curves
results_path = "runs/detect/yolov8m-advanced/results.png"
if os.path.exists(results_path):
    display(Image(filename=results_path))

# Confusion matrix
conf_matrix_path = "runs/detect/yolov8m-advanced/confusion_matrix.png"
if os.path.exists(conf_matrix_path):
    display(Image(filename=conf_matrix_path))

# ===============================
# 🔹 STEP 4: Test on Custom Images
# ===============================
test_images_path = f"{dataset_path}/test/images"
pred_results = model.predict(
    source=test_images_path,
    conf=0.25,   # confidence threshold
    save=True    # saves in runs/detect/predict
)

# Show one prediction
import glob
predict_dirs = glob.glob("runs/detect/predict*")
if predict_dirs:
    latest_predict_dir = max(predict_dirs, key=os.path.getctime)
    pred_images = glob.glob(f"{latest_predict_dir}/*.jpg")
    if pred_images:
        display(Image(filename=pred_images[0]))



Ultralytics 8.3.193 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Mineral-Identification-8/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m-advanced, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=Tr

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/yolov8m-advanced/weights/best.pt'

In [6]:
# ===============================
# 🔹 STEP 5: Save & Download Model
# ===============================
# Best weights
import shutil, os


# Create folder inside Drive to store models
drive_model_dir = "/content/drive/MyDrive/MineralDetectionModels/yolov8m-advanced"
os.makedirs(drive_model_dir, exist_ok=True)

# ===============================
# 🔹 STEP 6: Copy Best Model to Drive
# ===============================
# Path of YOLO saved weights in Colab
colab_model_path = "runs/detect/yolov8m-advanced/weights/best.pt"

# Copy best.pt into Drive
if os.path.exists(colab_model_path):
  shutil.copy(colab_model_path, drive_model_dir)
  print(f"✅ Best model saved to Google Drive at: {drive_model_dir}/best.pt")
else:
  print(f"❌ Best model weights not found at: {colab_model_path}. Training may have been interrupted.")

❌ Best model weights not found at: runs/detect/yolov8m-advanced/weights/best.pt. Training may have been interrupted.


In [7]:
import glob

# Search for all best.pt files
best_weights = glob.glob("runs/detect/**/best.pt", recursive=True)
print("Found models:", best_weights)


Found models: []


In [8]:
import shutil
from google.colab import files

# Path to your best model
model_path = "mineral-detection/yolov8m-advanced/weights/best.pt"

# Zip it for download
shutil.make_archive("best_model", 'zip', "mineral-detection/yolov8m-advanced/weights")

# Download zip file to your local system
files.download("best_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>